## Case Study 3 (CS3) - Sill Complex, Northwestern Australia
### Model D - Built using our adaption of the ODSIM (Henrion et al. 2008,2010) and structural frames (Grose et al. 2021).
- Considers a sample of approximately a 0.1% of the original dataset (Koepping et al. 2021). These points were classified depending on their location as top, base and lateral contacts. 
- Compared to Model C, Model D considers the emplacement model proposed by Koepping et al. (2021). The emplacement model is simplified as two sill propagating to the north and south of a central fault (interpreted as the feeder). Transgresive outer sills are controled by marginal faults.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Loop library
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer 

In [ ]:
lower_extent = [189000, 7775000, -12000]
upper_extent = [196350, 7790050, -8500]

#### 1. Load data

##### Input DataFrame description
> feature_name = name of the geological feature to be modelled. Faults 3 and 6 are the marginal faults that facilitated the emplacement of the inward-dipping outer sills

> (X, Y, Z) = data points location

> coord = structural frame coordinate

> val = values of the scalar field for interpolation of geological features

> (gx, gy, gz) = gradients of structural frame scalar fields

> intrusion_contact_type = roof/top or floor/base

> intrusion_side = TRUE if lateral contact, blank if not

> intrusion_anisotropy = name of geological feature in contact with intrusion (i.e. fault or stratigraphic unit)

In [ ]:
model_data = pd.read_csv('S3 Sill - Model D.csv')
northern_sill_data = model_data[model_data['feature_name'] == 'S3 northern sill'].copy()
southern_sill_data = model_data[model_data['feature_name'] == 'S3 southern sill'].copy()

northern_sill_inflation_data = model_data[(model_data['feature_name'] == 'S3 northern sill frame') & (model_data['coord'] == 0)].copy()
northern_sill_propagation_data = model_data[(model_data['feature_name'] == 'S3 northern sill frame') & (model_data['coord'] == 1)].copy()

southern_sill_inflation_data = model_data[(model_data['feature_name'] == 'S3 southern sill frame') & (model_data['coord'] == 0)].copy()
southern_sill_propagation_data = model_data[(model_data['feature_name'] == 'S3 southern sill frame') & (model_data['coord'] == 1)].copy()

model_data.head()

#### 2. Define conceptual model function to constraint lateral and vertical extent

These conceptual models are used to constraint the simulation of the intrusion lateral and vertical extent.

In [ ]:
def ellipse_function(
    lateral_contact_data, 
    minP=None, 
    maxP=None, 
    minS=None, 
    maxS=None
):

    if minP == None:
        minP = lateral_contact_data["coord1"].min()
    if maxP == None:
        maxP = lateral_contact_data["coord1"].max()
    if minS == None:
        minS = lateral_contact_data["coord2"].abs().min()
    if maxS == None:
        maxS = lateral_contact_data["coord2"].max()

    a = (maxP - minP) / 2
    b = (maxS - minS) / 2

    po = minP + (maxP - minP) / 2

    p_locations = lateral_contact_data.loc[:, "coord1"].copy().to_numpy()

    s = np.zeros([len(p_locations), 2])
    
    s[np.logical_and(p_locations>minP, p_locations<maxP),0] =  b * np.sqrt(1 - np.power((p_locations[np.logical_and(p_locations>minP, p_locations<maxP)] - po) / a, 2)) 
    s[np.logical_and(p_locations>minP, p_locations<maxP),1] =  -b * np.sqrt(1 - np.power((p_locations[np.logical_and(p_locations>minP, p_locations<maxP)] - po) / a, 2)) 
    
    return s

def parallelepiped_function(
    othercontact_data,
    mean_growth=None,
    minP=None,
    maxP=None,
    minS=None,
    maxS=None,
    vertex=None,
):

    if mean_growth == None:
        mean_growth = othercontact_data.loc[:, "coord0"].mean()

    data_ps = np.array(
        [othercontact_data.loc[:, "coord1"], othercontact_data.loc[:, "coord2"]]
    ).T

    conceptual_growth = np.ones([len(data_ps), 2]) * mean_growth

    return conceptual_growth

#### 3. Create geological model

##### The ``create_and_add_intrusion`` function of the ``GeologicalModel`` class creates an ``IntrusionFeature`` in two steps: 

**1. Create the intrusion frame using the ``IntrusionFrameBuilder`` class:**

IntrusionFrameBuilder creates the intrusion structural frame.
This object is a curvilinear coordinate system of the intrusion body. The extent of the intrusion is defined as distances along the axes of the intrusion frame.
The intrusion frame is constrained using the intrusion network points - a set of points representing the intrusion roof's location (or floor) contact - and propagation and inflation measurements.

**2. Simulate threshold distances to constrain the intrusion extent, using the ``IntrusionBuilder`` class:**

Simulation of thresholds distances (along structural frame coordinates) to constrain the intrusion lateral and vertical extent.** 
Distances along axis L are simulated to constraint the intrusion lateral extent and distances along axis G are simulated to constained its vertical extent. 
Simulated values are restricted using conceptual geometrical models representing the expected shape of the intrusion.
This step is performed using the GeostatPy library (https://pypi.org/project/geostatspy/), and its implementation of sequential gaussian simulation. 


##### The input parameters for the ``create_and_add_intrusion`` function are: 

> intrusion_name :  string, name of intrusion feature in model data

> intrusion_frame_name :  string, name of intrusion frame in model data
        
> intrusion_lateral_extent_model = function, geometrical conceptual model for simulation of lateral extent
        
> intrusion_vertical_extent_model = function, geometrical conceptual model for simulation of vertical extent
        
> intrusion_network_parameters : dictionary, optional. Keys:

        > contact : string, contact of the intrusion to be used to create the network (roof or floor, default: floor)
            
        > type : string, type of algorithm to create the intrusion network (interpolated or shortest path, default: interpolated)
                Shortest path is recommended when intrusion contact is not well constrained
        
        > contacts_anisotropies : list of series-type features involved in intrusion emplacement
        
        > structures_anisotropies : list of fault-type features involved in intrusion emplacement
        
        > sequence_anisotropies : list of anisotropies to look for the shortest path. It could be only starting and end point.
        
        > shortest_path_axis : string (X, Y or Z), shortest path is look within sections along this axis. 
                If faults presents, it is recommended this inext being orthogonal to the fault strike.

        > number_contacts : number of stratigraphic contacts from the same series  

        > delta_c, delta_f : deault values = 1. Used to identify the anisotropies using indicator functions. 
                Each anisotropy is characterized by an unknown isovalue of a scalar field *f*. 
                Observation points are evaluated on each scalar field, and we used the mean and standard deviation to approximate this isovalue. 
                An anisotropy is indicated in point (x,y,z) if 

                                    *(mean - std*delta)<=f(x,y,z)<=(mean + std*delta)*
        
> lateral_extent_sgs_parameters and vertical_extent_sgs_parameters = dictionary, optional, parameters for sequential gaussian simulation of lateral/vertical extent. 


        > tmin, tmax : all values, regardless of which variable, strictly less than tmin and greater than or equal to tmax are ignored.

                tmin = -9999 and tmax = 9999

        > itrans : 0 - no transformation requiered, data already nscored / 1 - for transformation.

                itrans = 1

        > ktype : type of interpolation, 0 for simple kriging - 1 for ordinary kriging.

                ktype = 0

        > nx, ny : Numbers of blocks. Grid node indices ix, iy increase from 1 to nx, ny respectively (in the positive x,y direction).

                nx, ny = nsteps[:,1]

        > xmn, ymn :  Centre of the first block (xmn,ymn).

                xmn, ymn = minimum coordinate p and l within the model boundaries

        > xsiz, ysiz : Size of blocks.

                xsiz, ysiz = model boundaries/nteps        

        > zmin, zmax, zmin2, zmax2 : The minimum and maximum allowable data values simulation.

                zmin, zmax = min and max of the input data for simulations

        > nxdis, nydis : Number of discretization points for a block. If both nxdis and nydis are set to 1, then point kriging is performed. 

                nxdis, nydis = 1

        > ndmin, ndmax : The minimum and maximum number of original data that should be used to simulate a grid node. If there are fewer than ndmin data points, the node is not simulated.

                ndmin = 0
                ndmax = 3

        > radius : The maximum isotropic search radius.

                radius = 500

        The default variogram is an isotropic variogram with infinite range, and no nugget effect. 

        > nug: nugget effect

                nug = 0

        > nst: number of nested structures (up to 2)

                nst = 1

        > it1, it2: nested structure 1,2 type (1 - spherical, 2 - exponential, 3 - Gaussian)

                it1 = 1, it2 = None

        > cc1,cc2: contribution to the sill of nested structure 1,2

                cc1 = 1, cc2 = None

        > azi1, azi2: azimuth of nested structure 1,2

                azi1 = 90, azi2 = None

        > hmaj1, hmin1, hmaj2, hmin2 : range of structure 1 in the major (hmaj1) and minor (hmin1) direction

                hmaj1, hmin1 = 999999, hmaj2, hmin2 = None


In [ ]:
#functions for visualization of faults

def fault3_extent(points):
    # points: array ([x,y,z])
    # maximum y value is scaled to model
    mask = np.zeros(len(points)).astype(bool)
    mask[points[:,1]< 9424.15] = 1
    
    return mask

def fault7_extent(points):
    # points: array ([x,y,z])
    # minimum and maximum x values are scaled to model
    mask1 = np.zeros(len(points)).astype(bool)
    mask2 = np.zeros(len(points)).astype(bool)
    mask1[points[:,0]> 3400.75] = 1
    mask2[points[:,0]< 6924.82] = 1
    return mask1 * mask2

In [ ]:
# # Create Geological Model of pre-intrusion geology

model = GeologicalModel(lower_extent,upper_extent)
displacement = 0
model.nsteps = [50,50,150]
model.data = model_data

fault3 = model.create_and_add_fault('fault_3',displacement,nelements=2000,steps=1,interpolatortype='PLI',buffer=0.3)
fault6 = model.create_and_add_fault('fault_6',displacement,nelements=2000,steps=1,interpolatortype='PLI',buffer=0.3)
fault7 = model.create_and_add_fault('fault_7',displacement,nelements=2000,steps=1,interpolatortype='PLI',buffer=0.3)

conformable_feature = model.create_and_add_foliation('stratigraphy', nelements = 3000, solver = 'lu', interpolatortype = 'FDI')

# Visualization of pre-intrusion geology

viewer_geo = LavaVuModelViewer(model, background='white')

# add stratigraphy
viewer_geo.add_isosurface(conformable_feature['feature'], isovalue = 0, colour = 'grey')

# add faults
viewer_geo.add_isosurface(fault3[0], isovalue = 0, colour = 'blue', region = fault3_extent)
viewer_geo.add_isosurface(fault6[0], isovalue = 0, colour = 'deepblue')
viewer_geo.add_isosurface(fault7[0], isovalue = 0, colour = 'lightblue',region = fault7_extent)

viewer_geo.add_points(northern_sill_data.loc[:,['X','Y','Z']].to_numpy(), name ='northern sill contact data', colour = 'black', pointsize = 6)
viewer_geo.add_points(southern_sill_data.loc[:,['X','Y','Z']].to_numpy(), name ='southern sill contact data', colour = 'black', pointsize = 6)

viewer_geo.add_points(northern_sill_inflation_data.loc[:,['X','Y','Z']].to_numpy(), name ='northern sill inflation data', colour = 'darkyellow', pointsize = 6)
viewer_geo.add_points(northern_sill_propagation_data.loc[:,['X','Y','Z']].to_numpy(), name ='northern sill propagation data', colour = 'purple', pointsize = 6)

viewer_geo.add_points(southern_sill_inflation_data.loc[:,['X','Y','Z']].to_numpy(), name ='southern sill inflation data', colour = 'darkyellow', pointsize = 6)
viewer_geo.add_points(southern_sill_propagation_data.loc[:,['X','Y','Z']].to_numpy(), name ='southern sill propagation data', colour = 'purple', pointsize = 6)

viewer_geo.interactive()

In [ ]:
# Add intrusions

intrusion_network_parameters = {'type' : 'shortest path' , 'contact' :'floor', 
                                'contact_anisotropies' : [conformable_feature], 'structures_anisotropies': [fault6],
                                'shortest_path_sequence':[conformable_feature, fault6],
                                'shortest_path_axis':'Y',
                                'delta_c' : [.1],
                                'delta_f' : [.01]
                               }

Northern_Sill = model.create_and_add_intrusion('S3 northern sill', intrusion_frame_name = 'S3 northern sill frame',
                                               intrusion_lateral_extent_model = ellipse_function,
                                               intrusion_vertical_extent_model = parallelepiped_function,
                                               vertical_extent_sgs_parameters = {'radius' : 5000, 'ndmin' : 10, 'ndmax' : 20},
                                               lateral_extent_sgs_parameters = {'radius' : 5000},
                                               intrusion_network_parameters = intrusion_network_parameters)

intrusion_network_parameters = {'type' : 'shortest path' , 'contact' :'floor', 
                                'contact_anisotropies' : [conformable_feature], 'structures_anisotropies': [fault3, fault6],
                                'shortest_path_sequence':[fault3, conformable_feature, fault6],
                                'shortest_path_axis':'Y',
                                'delta_c' : [1],
                                'delta_f' : [10,1],
                              
                               }

Southern_Sill = model.create_and_add_intrusion('S3 southern sill', intrusion_frame_name = 'S3 southern sill frame',
                                               intrusion_lateral_extent_model = ellipse_function,
                                               intrusion_vertical_extent_model = parallelepiped_function,
                                               vertical_extent_sgs_parameters = {'radius' : 5000, 'ndmin' : 10, 'ndmax' : 20},
                                               lateral_extent_sgs_parameters = {'radius' : 5000},
                                               intrusion_network_parameters = intrusion_network_parameters)

#### 4. Visualization of lateral and vertical extent simulation, along the intrusion frame coordinates

In [ ]:
intrusion = Northern_Sill

plt.subplot(221) # Lateral extent simulated values and data     
plt.plot(intrusion.lateral_simulated_thresholds.loc[:,'coord1'],intrusion.lateral_simulated_thresholds.loc[:,'min_l_threshold'],"-",c='orange', markersize = 10)
plt.plot(intrusion.lateral_simulated_thresholds.loc[:,'coord1'],intrusion.lateral_simulated_thresholds.loc[:,'max_l_threshold'],"-", label = 'simulated', c='darkorange', markersize = 10)
plt.plot(intrusion.lateral_simulated_thresholds.loc[:,'coord1'],intrusion.lateral_simulated_thresholds.loc[:,'conceptual_minl'],"-", c='green', markersize = 10)
plt.plot(intrusion.lateral_simulated_thresholds.loc[:,'coord1'],intrusion.lateral_simulated_thresholds.loc[:,'conceptual_maxl'],"-",  label = 'conceptual', c='green', markersize = 10)
plt.plot(intrusion.lateral_sgs_input_data[0].loc[:,'coord1'],intrusion.lateral_sgs_input_data[0].loc[:,'coord2'],"*",  label = 'data points', c='red', markersize = 10)
plt.plot(intrusion.lateral_sgs_input_data[1].loc[:,'coord1'],intrusion.lateral_sgs_input_data[1].loc[:,'coord2'],"*", c='red', markersize = 10)
plt.xlabel('Coordinate P'); plt.ylabel('Coordinate L'); plt.title('Sill 0 - Lateral extent simulated values and data')
plt.legend()
plt.grid(True)


plt.subplot(222) # Residual values  
plt.plot(intrusion.lateral_sgs_input_data[0].loc[:,'coord1'],intrusion.lateral_sgs_input_data[0].loc[:,'l_residual'],"*",  label = 'sides1', c='orange', markersize = 10)
plt.plot(intrusion.lateral_sgs_input_data[1].loc[:,'coord1'],intrusion.lateral_sgs_input_data[1].loc[:,'l_residual'],"*",  label = 'sides1', c='gold', markersize = 10)
plt.xlabel('Coordinate P'); plt.ylabel('Coordinate L'); plt.title('Sill 0 - Lateral extent residual values')
plt.grid(True)

plt.subplot(223) # Vertical extent     
vmin = intrusion.growth_simulated_thresholds.loc[:,'g_maximum'].min()
vmax = intrusion.growth_simulated_thresholds.loc[:,'g_maximum'].max()
plt.scatter(intrusion.growth_simulated_thresholds.loc[:,'coord1'],intrusion.growth_simulated_thresholds.loc[:,'coord2'], c=intrusion.growth_simulated_thresholds.loc[:,'g_maximum'])
plt.scatter(intrusion.vertical_sgs_input_data[1].loc[:,'coord1'],intrusion.vertical_sgs_input_data[1].loc[:,'coord2'])
plt.xlabel('Coordinate P'); plt.ylabel('Coordinate L'); plt.title('Sill 0 - Growth thresholds - Roof contact')
plt.grid(True)
plt.clim(vmin,vmax)
plt.colorbar()

plt.subplot(224)
vmin = intrusion.growth_simulated_thresholds.loc[:,'g_minimum'].min()
vmax = intrusion.growth_simulated_thresholds.loc[:,'g_minimum'].max()
plt.scatter(intrusion.growth_simulated_thresholds.loc[:,'coord1'],intrusion.growth_simulated_thresholds.loc[:,'coord2'], c=intrusion.growth_simulated_thresholds.loc[:,'g_minimum'])
plt.scatter(intrusion.vertical_sgs_input_data[0].loc[:,'coord1'],intrusion.vertical_sgs_input_data[0].loc[:,'coord2'])
plt.scatter(intrusion.vertical_sgs_input_data[1].loc[:,'coord1'],intrusion.vertical_sgs_input_data[1].loc[:,'coord2'])
plt.xlabel('Coordinate P'); plt.ylabel('Coordinate L'); plt.title('Sill 0 - Growth thresholds - Floor contact')
plt.grid(True)
plt.clim(vmin,vmax)
plt.colorbar()

plt.subplots_adjust(left=0.0, bottom=0.0, right=3, top=2, wspace=0.2, hspace=0.3)
plt.show()

In [ ]:
intrusion = Southern_Sill

plt.subplot(221) # Lateral extent simulated values and data     
plt.plot(intrusion.lateral_simulated_thresholds.loc[:,'coord1'],intrusion.lateral_simulated_thresholds.loc[:,'min_l_threshold'],"-",c='orange', markersize = 10)
plt.plot(intrusion.lateral_simulated_thresholds.loc[:,'coord1'],intrusion.lateral_simulated_thresholds.loc[:,'max_l_threshold'],"-", label = 'simulated', c='darkorange', markersize = 10)
plt.plot(intrusion.lateral_simulated_thresholds.loc[:,'coord1'],intrusion.lateral_simulated_thresholds.loc[:,'conceptual_minl'],"-", c='green', markersize = 10)
plt.plot(intrusion.lateral_simulated_thresholds.loc[:,'coord1'],intrusion.lateral_simulated_thresholds.loc[:,'conceptual_maxl'],"-",  label = 'conceptual', c='green', markersize = 10)
plt.plot(intrusion.lateral_sgs_input_data[0].loc[:,'coord1'],intrusion.lateral_sgs_input_data[0].loc[:,'coord2'],"*",  label = 'data points', c='red', markersize = 10)
plt.plot(intrusion.lateral_sgs_input_data[1].loc[:,'coord1'],intrusion.lateral_sgs_input_data[1].loc[:,'coord2'],"*", c='red', markersize = 10)
plt.xlabel('Coordinate P'); plt.ylabel('Coordinate L'); plt.title('Sill 0 - Lateral extent simulated values and data')
plt.legend()
plt.grid(True)


plt.subplot(222) # Residual values  
plt.plot(intrusion.lateral_sgs_input_data[0].loc[:,'coord1'],intrusion.lateral_sgs_input_data[0].loc[:,'l_residual'],"*",  label = 'sides1', c='orange', markersize = 10)
plt.plot(intrusion.lateral_sgs_input_data[1].loc[:,'coord1'],intrusion.lateral_sgs_input_data[1].loc[:,'l_residual'],"*",  label = 'sides1', c='gold', markersize = 10)
plt.xlabel('Coordinate P'); plt.ylabel('Coordinate L'); plt.title('Sill 0 - Lateral extent residual values')
plt.grid(True)

plt.subplot(223) # Vertical extent     
vmin = intrusion.growth_simulated_thresholds.loc[:,'g_maximum'].min()
vmax = intrusion.growth_simulated_thresholds.loc[:,'g_maximum'].max()
plt.scatter(intrusion.growth_simulated_thresholds.loc[:,'coord1'],intrusion.growth_simulated_thresholds.loc[:,'coord2'], c=intrusion.growth_simulated_thresholds.loc[:,'g_maximum'])
plt.scatter(intrusion.vertical_sgs_input_data[1].loc[:,'coord1'],intrusion.vertical_sgs_input_data[1].loc[:,'coord2'])
plt.xlabel('Coordinate P'); plt.ylabel('Coordinate L'); plt.title('Sill 0 - Growth thresholds - Roof contact')
plt.grid(True)
plt.clim(vmin,vmax)
plt.colorbar()

plt.subplot(224)
vmin = intrusion.growth_simulated_thresholds.loc[:,'g_minimum'].min()
vmax = intrusion.growth_simulated_thresholds.loc[:,'g_minimum'].max()
plt.scatter(intrusion.growth_simulated_thresholds.loc[:,'coord1'],intrusion.growth_simulated_thresholds.loc[:,'coord2'], c=intrusion.growth_simulated_thresholds.loc[:,'g_minimum'])
plt.scatter(intrusion.vertical_sgs_input_data[0].loc[:,'coord1'],intrusion.vertical_sgs_input_data[0].loc[:,'coord2'])
plt.scatter(intrusion.vertical_sgs_input_data[1].loc[:,'coord1'],intrusion.vertical_sgs_input_data[1].loc[:,'coord2'])
plt.xlabel('Coordinate P'); plt.ylabel('Coordinate L'); plt.title('Sill 0 - Growth thresholds - Floor contact')
plt.grid(True)
plt.clim(vmin,vmax)
plt.colorbar()

plt.subplots_adjust(left=0.0, bottom=0.0, right=3, top=2, wspace=0.2, hspace=0.3)
plt.show()

#### 5. Visualization of 3D Geological Model

In [ ]:
viewer = LavaVuModelViewer(model, background='white')
# viewer.nsteps = [40,40,150]

viewer.add_points(northern_sill_data.loc[:,['X','Y','Z']].to_numpy(), name = 'Northern sill data', pointsize = 5)
viewer.add_points(southern_sill_data.loc[:,['X','Y','Z']].to_numpy(), name = 'Southern sill data', pointsize = 5)

# # add stratigraphy
viewer.add_isosurface(conformable_feature['feature'], isovalue = 0, colour = 'grey')

# # add faults
viewer.add_isosurface(fault3[0], isovalue = 0, colour = 'blue')
viewer.add_isosurface(fault6[0], isovalue = 0, colour = 'deepblue')
viewer.add_isosurface(fault7[0], isovalue = 0, colour = 'lightblue')

# add intrusion
viewer.add_isosurface(Northern_Sill, isovalue = 0, colour = 'red')
viewer.add_isosurface(Southern_Sill, isovalue = 0, colour = 'red')

viewer.interactive()

#### 6. Visualization of intrusion networks and intrusion frames

In [ ]:
Northern_Sill_inet = Northern_Sill.intrusion_frame.builder.intrusion_network_points
Southern_Sill_inet = Southern_Sill.intrusion_frame.builder.intrusion_network_points

Northern_Sill_frame = Northern_Sill.intrusion_frame
Southern_Sill_frame = Southern_Sill.intrusion_frame

In [ ]:
viewer_nsill = LavaVuModelViewer(model, background='white')

viewer_nsill.add_points(model.rescale(Northern_Sill_inet[:,:3], inplace = False), name ='Northern Sill Intrusion Network', colour = 'blue', pointsize = 6)

viewer_nsill.add_isosurface(Northern_Sill_frame[0], isovalue = 0, colour = 'darkred')
viewer_nsill.add_isosurface(Northern_Sill_frame[1], isovalue = 0, colour = 'darkgreen')
viewer_nsill.add_isosurface(Northern_Sill_frame[2], isovalue = 0, colour = 'blue')
viewer_nsill.add_isosurface(fault6[0], isovalue = 0, colour = 'deepblue')
viewer_nsill.add_data(Northern_Sill_frame[0])

viewer_nsill.interactive()

In [ ]:
viewer_ssill = LavaVuModelViewer(model, background='white')

viewer_ssill.add_points(model.rescale(Southern_Sill_inet[:,:3], inplace = False), name ='Southern Sill Intrusion Network', colour = 'green', pointsize = 6)

viewer_ssill.add_points(model.rescale(southern_sill_indicator_fxs_F[:,:3], inplace = False), name ='IFF', colour = 'blue', pointsize = 6)
viewer_ssill.add_points(model.rescale(southern_sill_indicator_fxs_C[:,:3], inplace = False), name ='IFC', colour = 'red', pointsize = 6)

viewer_ssill.add_isosurface(Southern_Sill_frame[0], isovalue = 0, colour = 'darkred')
viewer_ssill.add_isosurface(Southern_Sill_frame[1], isovalue =0, colour = 'darkgreen')
viewer_ssill.add_isosurface(Southern_Sill_frame[2], isovalue = 0, colour = 'blue')

viewer_ssill.add_data(Southern_Sill_frame[0])
viewer_ssill.add_data(Southern_Sill_frame[1])

viewer_ssill.interactive()